In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import random


In [3]:
movie_df = pd.read_csv('./df_movies_final.csv')
ratings_df = pd.read_csv('./ratings_without_timestamp.csv')

In [4]:
AllRatings = ratings_df

In [5]:
AllRatings_sub = AllRatings[AllRatings['userId'] <= 5000]
AllRatings_sub

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
733150,5000,1073,2.0
733151,5000,1210,3.0
733152,5000,1356,3.0
733153,5000,1393,4.0


In [6]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [7]:
# ratings_size = AllRatings
AllRatings_sub = AllRatings_sub.sample(frac=1, random_state=42).reset_index(drop=True)
AllRatings_sub = AllRatings_sub.to_numpy()
ratingsTrain = AllRatings_sub[:int(len(AllRatings_sub)*0.6)]
ratingsValid = AllRatings_sub[int(len(AllRatings_sub)*0.6):int(len(AllRatings_sub)*0.8)]
ratingsTest = AllRatings_sub[int(len(AllRatings_sub)*0.8):]

usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
ratingOnly = [] # to calculate global average
ratingDict = {}
userID, itemID = set(), set()

for user, item, rating in ratingsTrain:
    user, item, rating = str(int(user)), str(int(item)), rating
    userID.add(user)
    itemID.add(item)

    ratingsPerUser[user].append((item,rating))
    ratingsPerItem[item].append((user,rating))
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = rating
    ratingOnly.append(rating)



In [ ]:
# Grace version

def preprocess_data(ratingsTrain):
    user_ratings = defaultdict(list)
    item_ratings = defaultdict(list)
    all_ratings = []
    for u, i, r in ratingsTrain:
        rating = int(r)
        user_ratings[u].append((i, rating))
        item_ratings[i].append((u, rating))
        all_ratings.append(rating)
    return user_ratings, item_ratings, all_ratings

def compute_biases(user_ratings, item_ratings, global_average, lamb):
    """Compute user and item biases using gradient descent."""
    user_bias = {u: 0 for u in user_ratings}
    item_bias = {i: 0 for i in item_ratings}
    
    # Gradient descent for biases
    for iteration in range(50):  # Fixed number of iterations
        # Update user biases
        for u in user_ratings:
            user_bias[u] = sum(r - (global_average + item_bias[i]) for i, r in user_ratings[u]) / (
                lamb + len(user_ratings[u])
            )
        # Update item biases
        for i in item_ratings:
            item_bias[i] = sum(r - (global_average + user_bias[u]) for u, r in item_ratings[i]) / (
                lamb + len(item_ratings[i])
            )
    return user_bias, item_bias

def rating_prediction(ratingsValid, lamb=5.0):
    """Predict ratings using user and item biases."""
    user_ratings, item_ratings, all_ratings = preprocess_data(ratingsTrain)
    global_average = np.mean(all_ratings)
    user_bias, item_bias = compute_biases(user_ratings, item_ratings, global_average, lamb)

    pred_rating = []
    true_rating = []
    for u, i, r in ratingsValid:
        p = global_average + user_bias.get(u, 0) + item_bias.get(i, 0)
        true_rating.append(r)
        pred_rating.append(p)
    mse = MSE(pred_rating, true_rating)
    print(mse)

rating_prediction(ratingsValid)



0.7738235360013701


In [ ]:
# def preprocess_data(train_path):
#     """Preprocess data to extract user and item biases."""
#     user_ratings = defaultdict(list)
#     item_ratings = defaultdict(list)
#     all_ratings = []
    
#     # Collect user-item pairs with ratings
#     for user, book, rating in readCSV(train_path):
#         rating = int(rating)
#         user_ratings[user].append((book, rating))
#         item_ratings[book].append((user, rating))
#         all_ratings.append(rating)
    
#     return user_ratings, item_ratings, all_ratings

# user_ratings = ratingsPerUser
# item_ratings = ratingsPerItem
# all_ratings = ratingOnly


In [ ]:
def compute_biases(user_ratings, item_ratings, global_average, lamb):
    """Compute user and item biases using gradient descent."""
    user_bias = {u: 0 for u in user_ratings}
    item_bias = {i: 0 for i in item_ratings}
    
    # Gradient descent for biases
    for iteration in range(50):  # Fixed number of iterations
        # Update user biases
        for u in user_ratings:
            user_bias[u] = sum(r - (global_average + item_bias[i]) for i, r in user_ratings[u]) / (
                lamb + len(user_ratings[u])
            )
        # Update item biases
        for i in item_ratings:
            item_bias[i] = sum(r - (global_average + user_bias[u]) for u, r in item_ratings[i]) / (
                lamb + len(item_ratings[i])
            )
         
    return user_bias, item_bias

In [18]:
# def rating_prediction(train_path, test_path, output_path, lamb=5.0):
def rating_prediction(ratingsValid, lamb=5.0):

    """Predict ratings using user and item biases."""
    user_ratings, item_ratings, all_ratings = ratingsPerUser, ratingsPerItem, ratingOnly
    global_average = np.mean(all_ratings)
    user_bias, item_bias = compute_biases(user_ratings, item_ratings, global_average, lamb)

    # # Predict on the test set
    # with open(output_path, 'w') as predictions:
    #     predictions.write("userID,bookID,prediction\n")
    #     for line in open(test_path):
    #         if line.startswith("userID"):
    #             continue
    #         user, book = line.strip().split(',')
    #         pred = global_average + user_bias.get(user, 0) + item_bias.get(book, 0)
    #         predictions.write(f"{user},{book},{pred:.2f}\n")
    pred_rating = []
    true_rating = []
    for u, i, r in ratingsValid:
        p = global_average + user_bias.get(u, 0) + item_bias.get(i, 0)
        true_rating.append(r)
        pred_rating.append(p)
    mse = MSE(pred_rating, true_rating)
    print(mse)

In [ ]:
def tune_lambda(train_path, test_path, validation_path):
    """Find the optimal lambda value for rating prediction."""
    best_lambda = None
    best_mse = float('inf')
    for lamb in np.arange(1.0, 10.1, 0.5):
        output_path = "predictions_Validation.csv"
        rating_prediction(train_path, validation_path, output_path, lamb=lamb)
        true_ratings = [int(line.strip().split(',')[2]) for line in open(validation_path) if not line.startswith("userID")]
        pred_ratings = [float(line.strip().split(',')[2]) for line in open(output_path) if not line.startswith("userID")]
        mse = MSE(true_ratings, pred_ratings)
        if mse < best_mse:
            best_mse = mse
            best_lambda = lamb
    print(f"Best Lambda: {best_lambda}, Best MSE: {best_mse}")
    return best_lambda

In [16]:
def output_rating_predictions():
    train_path = "train_Interactions.csv.gz"
    test_path = "pairs_Rating.csv"
    predictions_path = "predictions_Rating.csv"
    rating_prediction(ratingsValid)

In [19]:
if __name__ == "__main__":
    output_rating_predictions()

1.1000496208108885


In [34]:
lambda_reg = 0.2
learning_rate = 0.01
n_epochs = 40
K = 5

best_mse = float('inf')
patience = 5
patience_counter = 0

# 
ratings_train = [rating for _, _, rating in ratingsTrain]
alpha = np.mean(ratings_train)

user_biases = defaultdict(float)
item_biases = defaultdict(float)

userGamma = defaultdict(lambda: [0.0] * K, {user: [random.random() * 0.1 - 0.05 for _ in range(K)] for user in ratingsPerUser})
itemGamma = defaultdict(lambda: [0.0] * K, {item: [random.random() * 0.1 - 0.05 for _ in range(K)] for item in ratingsPerItem})

for epoch in range(n_epochs):
    if epoch < 30:
        learning_rate = 0.01
    else:
        learning_rate = 0.001

    for user, item, rating in ratingsTrain:
        if user not in userGamma:
            userGamma[user] = [random.random() * 0.1 - 0.05 for _ in range(K)]
        if item not in itemGamma:
            itemGamma[item] = [random.random() * 0.1 - 0.05 for _ in range(K)]
        
        prediction = alpha + user_biases.get(user, 0) + item_biases.get(item, 0) + inner(userGamma[user], itemGamma[item])
        error = rating - prediction

        user_biases[user] += learning_rate * (error - lambda_reg * user_biases[user])
        item_biases[item] += learning_rate * (error - lambda_reg * item_biases[item])
        
        for k in range(K):
            userGamma[user][k] += learning_rate * (error * itemGamma[item][k] - lambda_reg * userGamma[user][k])
            itemGamma[item][k] += learning_rate * (error * userGamma[user][k] - lambda_reg * itemGamma[item][k])

    squared_errors = []
    for user, item, rating in ratingsValid:
        prediction = alpha + user_biases.get(user, 0) + item_biases.get(item, 0) + inner(userGamma.get(user, [0] * K), itemGamma.get(item, [0] * K))
        squared_errors.append((rating - prediction) ** 2)
    mse = np.mean(squared_errors)
    print(f"Epoch {epoch+1}/{n_epochs}, Validation MSE: {mse}")

    if mse < best_mse:
        best_mse = mse
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping...")
            break


Epoch 1/40, Validation MSE: 0.8095351285851702
Epoch 2/40, Validation MSE: 0.7833409579031452
Epoch 3/40, Validation MSE: 0.7724539950360055
Epoch 4/40, Validation MSE: 0.7665107008245938
Epoch 5/40, Validation MSE: 0.7628291333981808
Epoch 6/40, Validation MSE: 0.7603709818138574
Epoch 7/40, Validation MSE: 0.7586445599239138
Epoch 8/40, Validation MSE: 0.7573868074140805
Epoch 9/40, Validation MSE: 0.7564447957696249
Epoch 10/40, Validation MSE: 0.7557240183371043
Epoch 11/40, Validation MSE: 0.7551632406432471
Epoch 12/40, Validation MSE: 0.754721250169726
Epoch 13/40, Validation MSE: 0.7543694269527974
Epoch 14/40, Validation MSE: 0.754087363452561
Epoch 15/40, Validation MSE: 0.7538601730522678
Epoch 16/40, Validation MSE: 0.7536767778727188
Epoch 17/40, Validation MSE: 0.7535287870842289
Epoch 18/40, Validation MSE: 0.7534097432748597
Epoch 19/40, Validation MSE: 0.7533146048545979
Epoch 20/40, Validation MSE: 0.7532393836107224
Epoch 21/40, Validation MSE: 0.7531808864705776
Epo

,userId,movieId,rating
0,653,139299,2.5
1,3854,1466,5.0
2,4554,8360,3.5
3,84,6711,4.0
4,1866,90531,4.0
...,...,...,...
439888,2096,4993,4.0
439889,3554,5349,5.0
439890,85,21,4.0
439891,2379,4356,3.5
